---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-text-mining/resources/d9pwm) course resource._

---

# Assignment 1

In this assignment, you'll be working with messy medical data and using regex to extract relevant infromation from the data. 

Each line of the `dates.txt` file corresponds to a medical note. Each note has a date that needs to be extracted, but each date is encoded in one of many formats.

The goal of this assignment is to correctly identify all of the different date variants encoded in this dataset and to properly normalize and sort the dates. 

Here is a list of some of the variants you might encounter in this dataset:
* 04/20/2009; 04/20/09; 4/20/09; 4/3/09
* Mar-20-2009; Mar 20, 2009; March 20, 2009;  Mar. 20, 2009; Mar 20 2009;
* 20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
* Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009
* Feb 2009; Sep 2009; Oct 2010
* 6/2008; 12/2009
* 2009; 2010

Once you have extracted these date patterns from the text, the next step is to sort them in ascending chronological order accoring to the following rules:
* Assume all dates in xx/xx/xx format are mm/dd/yy
* Assume all dates where year is encoded in only two digits are years from the 1900's (e.g. 1/5/89 is January 5th, 1989)
* If the day is missing (e.g. 9/2009), assume it is the first day of the month (e.g. September 1, 2009).
* If the month is missing (e.g. 2010), assume it is the first of January of that year (e.g. January 1, 2010).
* Watch out for potential typos as this is a raw, real-life derived dataset.

With these rules in mind, find the correct date in each note and return a pandas Series in chronological order of the original Series' indices.

For example if the original series was this:

    0    1999
    1    2010
    2    1978
    3    2015
    4    1985

Your function should return this:

    0    2
    1    4
    2    0
    3    1
    4    3

Your score will be calculated using [Kendall's tau](https://en.wikipedia.org/wiki/Kendall_rank_correlation_coefficient), a correlation measure for ordinal data.

*This function should return a Series of length 500 and dtype int.*

In [ ]:
import pandas as pd

doc = []
with open('dates.txt') as file:
    for line in file:
        doc.append(line)

df = pd.Series(doc)
#df.head(10)

In [7]:
def date_sorter():
    
    # Load the data
    import pandas as pd

    doc = []
    with open('dates.txt') as file:
        for line in file:
            doc.append(line)

    df = pd.Series(doc)
    
    # Match the various Date formats
    tmp1 = df.str.extractall(r'(?P<date>(?P<month>\d{1,2})[-/](?P<day>\d{1,2})[-/](?P<year>\d{2,4}))')
    tmp2 = df.str.extractall(r'(?P<date>(?P<month>Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*[ ,\.]{1,2}(?P<day>\d{1,2})[ ,\.]{1,2}(?P<year>\d{2,4}))')
    tmp3 = df.str.extractall(r'(?P<date>(?P<day>\d{2})[ ,\.-]+(?P<month>Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*[ , \.-]+(?P<year>\d{2,4}))')
    tmp4 = df.str.extractall(r'(?P<date>(?P<month>Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec) (?P<day>\d{1,2})[thsnd]{2}, (?P<year>\d{4}))')
    tmp5 = df.str.extractall(r'(?P<date>(?P<month>Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*[ ,](?P<year>\d{4}))')
    tmp6 = df.str.extractall(r'(?P<date>(?P<month>\d{1,2})/(?P<year>\d{4}))')
    tmp7 = df.str.extractall(r'(?P<date>(?P<year>[ ]*[1-2]\d{3}[ ]*))')
    
    # map the months into integers
    months_label = {'Jan': '01',
                    'Feb': '02',
                    'Mar': '03',
                    'Apr': '04',
                    'May': '05',
                    'Jun': '06',
                    'Jul': '07',
                    'Aug': '08',
                    'Sep': '09',
                    'Oct': '10',
                    'Nov': '11',
                    'Dec': '12'
    }
    tmp2['month'] = tmp2['month'].map(months_label)
    tmp3['month'] = tmp3['month'].map(months_label)
    tmp4['month'] = tmp4['month'].map(months_label)
    tmp5['month'] = tmp5['month'].map(months_label)
    
    # Add the missing columns to some of the data.frames
    tmp5['day'] = tmp6['day'] = tmp7['day'] = ''
    tmp7['month'] = ''
    
    # Concatenate the data.frames
    tmplist = [tmp2, tmp3, tmp4, tmp5, tmp6, tmp7]
    final = tmp1
    for tmp in tmplist:
        final = pd.concat([final, tmp[['date', 'year', 'month', 'day']]], axis = 0)
    final.reset_index(inplace=True)
    final = pd.DataFrame(final[['level_0', 'date', 'day', 'month', 'year']])
    final.set_index('level_0', inplace=True)
    final.index.name = 'index'
    final['num_missing_info'] = final.isnull().sum(axis=1)     
    
    # Let's handle duplicates
    final = final.merge(final
                        .groupby(final.index)['num_missing_info']
                        .min()
                        .rename('min_missing')
                        .to_frame(),
                        left_index = True,
                        right_index = True
                       )
    final = final[final['num_missing_info'] == final['min_missing']]
    
    # We have a duplicate. It's a year with 3 digits (wrong match).
    # Let's get rid of it
    final = final[final['year'].str.len() != 3]
    
    # Fill gaps
    final.replace('', '01', inplace=True)
    final.fillna(value='01', inplace=True)
    
    # Pad month, day, and year to get the proper string lengths
    final['day'] = final['day'].str.zfill(2)
    final['month'] = final['month'].str.zfill(2)
    final['year'] = final['year'].str.rjust(width = 3, fillchar = '9').str.rjust(width = 4, fillchar = '1')
    
    # Let's prepare the output
    final_out = (final['year']+final['month']+final['day']).str.replace(' ', '').astype(float)
    final_out.index = final_out.index.get_level_values(0)
    
    tmp_out = final_out.reset_index(level=[0]).groupby('index').max()
    tmp_out.columns = ['date']
    tmp_out.sort_values('date', inplace=True)
    
    # for debugging purposes
    final_df = tmp_out.merge(df.to_frame(), left_index = True, right_index = True).reset_index(level=[0])
    return final_df['index'].rename(None)

In [8]:
out = date_sorter()

/Applications/anaconda3/envs/testingenv/lib/python3.7/site-packages/ipykernel_launcher.py:49: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



In [9]:
out

0        9
1       84
2        2
3       53
4       28
5      474
6      153
7       13
8      129
9       98
10     111
11     225
12      31
13     171
14     191
15     486
16     335
17     415
18      36
19     323
20     405
21     422
22     375
23     380
24     345
25      57
26     481
27     436
28     104
29     299
      ... 
470    243
471    208
472    139
473    320
474     34
475    383
476    286
477    244
478    480
479    431
480    279
481    198
482    381
483    463
484    366
485    439
486    255
487    401
488    475
489    257
490    152
491    235
492    464
493    253
494    427
495    231
496    141
497    186
498    161
499    413
Length: 500, dtype: int64